In [9]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [10]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

In [11]:
train_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=transform
)

In [12]:
test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=transform
)

In [13]:
len(train_data)

60000

In [14]:
len(test_data)

10000

In [15]:
train_loader = DataLoader(
    dataset=train_data,
    batch_size=64,
    shuffle=True
)

test_loader = DataLoader(
    dataset=test_data,
    batch_size=64,
    shuffle=True
)

In [16]:
images, labels = next(iter(train_loader))
print(images.shape)
print(labels.shape)
print(images.min(), images.max())

torch.Size([64, 1, 28, 28])
torch.Size([64])
tensor(-1.) tensor(1.)
